# Transformation von Daten

Nich alle Input-Daten können unbesehen verwendet werden. Es ist notwendig, kategorische Daten entweder in Zahlen umzuwandeln (ordinale kategorische Daten) oder über ein sogenanntes One-Hot-Encoding nominale kategorische Daten in sinnvolle Spalten umzuwaneln (s.u.). 

Transformation MÜSSEN wir machen, weil zb Holz, Asbest kann man nicht sortieren

In [1]:
#import some necessary librairies

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew #for some statistics


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points


from subprocess import check_output
print(check_output(["ls", "../data/house-prices-advanced-regression-techniques"]).decode("utf8")) #check the files available in the directory

import pickle

all_data.pkl
data_description.txt
test.csv
test.pkl
test_1.pkl
train.csv
train.pkl
train_1.pkl
x_preprocessed.pkl
x_preprocessed_train.pkl
x_test.pkl
y_train.pkl



# Daten aus dem vorigen Schritt laden


In [ ]:
with open('../data/house-prices-advanced-regression-techniques/train_1.pkl', 'rb') as handle:
    train_orig = pickle.load(handle)

with open('../data/house-prices-advanced-regression-techniques/test_1.pkl', 'rb') as handle:
    test_orig = pickle.load(handle)
    
with open('../data/house-prices-advanced-regression-techniques/all_data.pkl', 'rb') as handle:
    df = pickle.load(handle)

# weil wir haben ja cheaterig die zsm gemacht earlier

# Wiederherstellen von Test- und Trainset

In [7]:
#X ist Feature und y ist Zielwert


ntrain = train_orig.shape[0] # Anzahl der Trainingsdaten
ntest = test_orig.shape[0]   # Anzahl der Testdaten

# Zielvariable der Trainingsdaten
y_train = train_orig['SalePrice']

In [4]:
print (y_train)
print ("Länge von y_train: " + str (len(y_train)))

0      12.248
1      12.109
2      12.317
3      11.849
4      12.429
        ...  
1455   12.073
1456   12.255
1457   12.493
1458   11.864
1459   11.902
Name: SalePrice, Length: 1458, dtype: float64
Länge von y_train: 1458


In [ ]:
X_train = df[:ntrain]
X_test = df[ntrain:]

print("Dimension des Trainingssets: " + str (X_train.shape))
print("Dimension des Testsets: " + str (X_test.shape))


#haben also noch 74 spalten

Dimension des Trainingssets: (1458, 74)
Dimension des Testsets: (1459, 74)


## Nominale Daten

Die einfachste Transformation, von textbasierten zu numerischen kategorischen Merkmalen, ist das Ersetzen einer Kategorie mit einer bestimmten Zahl. Zum Beispiel für "Hund" die 1, für "Maus" die 2 und für "Katze" die 3. Einige ML-Algorithmen gehen jedoch davon aus, dass sich **zwei benachbarte Werte ähnlicher** sind als weiter entfernte Werte. Bei nominalen Daten trifft diese Annahme nicht zu. Es muss sicher gestellt werden, dass der Algorithmus hier **keine Fehlinterpretation** vornimmt. Lösung ist das sogenannte **One-Hot-Encoding**.  

### One-Hot-Encoding

Beim One-Hot-Encoding wird für **jeden möglichen Wert eine Spalte** mit binären Werten erstellt. 1 bedeutet, der Wert liegt vor, 0 bedeutet der Wert liegt nicht vor. Das Merkmal Tierart mit den drei möglichen Kategorien "Hund", "Maus" und "Katze" wird mit dem One-Hot-Encoding in drei Spalten transformiert. Für die Umsetzung bietet sich die [get_dummies()](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html)-Methode von Pandas an.

Das nachfolgende Beispiel zeigt dies für die nominal kategorische Spalte 'Electrical' (d.h. die Aussage, welche elektrische Installation im Haus vorzufinden ist).

Schauen Sie genau hin. Es ist in jeweils nur einer einzigen Spalte der Wert "True" zu sehen!

In [ ]:
pd.get_dummies(df['Electrical'])  # die nutzen wir nicht die funktion!

,FuseA,FuseF,FuseP,Mix,SBrkr
0,False,False,False,False,True
1,False,False,False,False,True
2,False,False,False,False,True
3,False,False,False,False,True
4,False,False,False,False,True
...,...,...,...,...,...
2912,False,False,False,False,True
2913,False,False,False,False,True
2914,False,False,False,False,True
2915,False,False,False,False,True


# Erstellen einer Pipeline
Um den Prozess der Verarbeitung zu vereinfachen und zu standardisieren, kann eine sogenannte Pipeline eingesetzt werden. Diese erlaubt sowohl die Skalierung von Daten als auch das One-Hot-Encoding.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# also jeder transformer hat quasi n inputter der die datne in empfang nimmt (also transfomer = pipelines)
# Define transformers for numerical and categorical columns
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output = False))
])

# Vgl. https://www.kaggle.com/code/kenjee/housing-prices-example-with-video-walkthrough

In [ ]:
# Update categorical and numerical columns
categorical_columns = df.select_dtypes(include=['object', 'category']).columns #'object also basically strings
numerical_columns = df.select_dtypes(include=[np.number]).columns

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ],remainder = 'passthrough')

# Create a pipeline with the preprocessor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)])

X_preprocessed_train = pipeline.fit_transform(X_train)

print ("Anzahl numerischer Spalten: " + str(len(numerical_columns)))
print ("Anzahl kategorischer Spalten: " + str(len(categorical_columns)))


Anzahl numerischer Spalten: 36
Anzahl kategorischer Spalten: 38


In [11]:
X_preprocessed_train

array([[ 0.07342554, -0.22286241, -0.20393408, ...,  0.        ,
         1.        ,  0.        ],
       [-0.87186754,  0.49025596, -0.08725172, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.07342554, -0.08023874,  0.08016212, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.30974881, -0.17532119, -0.14386803, ...,  0.        ,
         1.        ,  0.        ],
       [-0.87186754, -0.08023874, -0.07538055, ...,  0.        ,
         1.        ,  0.        ],
       [-0.87186754,  0.25254984, -0.05305871, ...,  0.        ,
         1.        ,  0.        ]], shape=(1458, 282))

In [ ]:
# Function to create scrollable table within a small window
def create_scrollable_table(df, table_id, title):
    html = f'<h3>{title}</h3>'
    html += f'<div id="{table_id}" style="height:200px; overflow:auto;">'
    html += df.to_html()
    html += '</div>'
    return html

In [ ]:
from IPython.display import display, HTML
html_preprocessed = create_scrollable_table(pd.DataFrame(X_preprocessed_train), 'numerical_features', 'Summary statistics for numerical features')
display(HTML(html_preprocessed))


In [ ]:
with open('../data/house-prices-advanced-regression-techniques/x_preprocessed_train.pkl', 'wb') as handle:
    pickle.dump(X_preprocessed_train, handle)
    
with open('../data/house-prices-advanced-regression-techniques/y_train.pkl', 'wb') as handle:
    pickle.dump(y_train, handle)
    
with open('../data/house-prices-advanced-regression-techniques/x_test.pkl', 'wb') as handle:
    pickle.dump(X_test, handle)
    
